In [153]:
import numpy as np
from random import randrange
from scipy.sparse import diags
# from scipy.sparse import lil_matrix as csr
from math import sqrt
from numpy.linalg import eigh


class MatrixGeneration:

    @staticmethod
    def tridiagonal(n):
        r = randrange(2, 10)
        k = [(r // 2) * np.ones(n - 1), r * np.ones(n), (r // 2) * np.ones(n - 1)]
        offset = [-1, 0, 1]
        return np.array(diags(k, offset).toarray(), dtype=float)

    @staticmethod
    def hilbert(n):
        return np.array([[1 / (i + j + 1) for i in range(n)] for j in range(n)], dtype=float)

In [168]:
class Jacobi:

    def __init__(self, a, eps=1e-9):
        self.a = a
        self.n = len(a)
        self.eps = eps

    def max_element_right(self):
        a_max, k, l = 0, 0, 0
        for i in range(self.n - 1):
            for j in range(i + 1, self.n):
                if abs(self.a[i, j]) >= a_max:
                    a_max, k, l = abs(self.a[i, j]), i, j
        return a_max, k, l

    def rotate(self, transform_matrix, k, l):
        a_diff = self.a[l, l] - self.a[k, k]
        if abs(self.a[k, l]) < abs(a_diff) * 1.0e-36:
            t = self.a[k, l] / a_diff
        else:
            phi = a_diff / (2 * self.a[k, l])
            t = 1.0 / (abs(phi) + sqrt(phi ** 2 + 1))
            if phi < 0:
                t = -t

        c = 1 / sqrt(t ** 2 + 1)
        s = t * c
        tau = s / (1 + c)
        tmp = self.a[k, l]
        self.a[k, l] = 0
        self.a[k, k] = self.a[k, k] - t * tmp
        self.a[l, l] = self.a[l, l] + t * tmp
        for i in range(k):  # i < k
            tmp = self.a[i, k]
            self.a[i, k] = tmp - s * (self.a[i, l] + tau * tmp)
            self.a[i, l] = self.a[i, l] + s * (tmp - tau * self.a[i, l])
        for i in range(k + 1, l):  # k < i < l
            tmp = self.a[k, i]
            self.a[k, i] = tmp - s * (self.a[i, l] + tau * self.a[k, i])
            self.a[i, l] = self.a[i, l] + s * (tmp - tau * self.a[i, l])
        for i in range(l + 1, self.n):  # i > l
            tmp = self.a[k, i]
            self.a[k, i] = tmp - s * (self.a[l, i] + tau * tmp)
            self.a[l, i] = self.a[l, i] + s * (tmp - tau * self.a[l, i])
        for i in range(self.n):  # updating transformation matrix
            tmp = transform_matrix[i, k]
            transform_matrix[i, k] = tmp - s * (transform_matrix[i, l] + tau * transform_matrix[i, k])
            transform_matrix[i, l] = transform_matrix[i, l] + s * (tmp - tau * transform_matrix[i, l])

    def run(self, max_rotations=None):
        max_rotations = 5 * (self.n ** 2) if max_rotations is None else max_rotations
        rotations = 0
        transform_matrix = np.eye(self.n)
        for i in range(max_rotations):
            a_max, k, l = self.max_element_right()
            if a_max < self.eps:
                return np.diagonal(self.a), rotations
            self.rotate(transform_matrix, k, l)
            rotations += 1
        print('Not converges')

### Check that method works correct:

In [169]:
matrix = MatrixGeneration.tridiagonal(6)
print(sorted(eigh(matrix)[0]))
print(sorted(Jacobi(matrix).run()[0]))

[0.7922490567806474, 3.012081585130133, 6.219832528349487, 9.780167471650515, 12.987918414869865, 15.207750943219354]
[0.792249056780647, 3.0120815851301317, 6.219832528349484, 9.780167471650515, 12.987918414869867, 15.207750943219352]


In [174]:
print([(n, Jacobi(MatrixGeneration.tridiagonal(n), eps=1e-3).run()[1]) for n in range(3, 50)])

[(3, 7), (4, 6), (5, 24), (6, 35), (7, 48), (8, 63), (9, 79), (10, 101), (11, 116), (12, 142), (13, 173), (14, 199), (15, 251), (16, 264), (17, 309), (18, 321), (19, 382), (20, 388), (21, 436), (22, 499), (23, 539), (24, 593), (25, 575), (26, 678), (27, 762), (28, 783), (29, 799), (30, 919), (31, 925), (32, 924), (33, 990), (34, 1172), (35, 1189), (36, 1265), (37, 1209), (38, 1262), (39, 1481), (40, 1416), (41, 1555), (42, 1759), (43, 1601), (44, 1657), (45, 1924), (46, 1964), (47, 2168), (48, 2266), (49, 2199)]


In [175]:
print([(n, Jacobi(MatrixGeneration.tridiagonal(n), eps=1e-6).run()[1]) for n in range(3, 50)])

[(3, 8), (4, 6), (5, 29), (6, 31), (7, 61), (8, 77), (9, 108), (10, 129), (11, 162), (12, 176), (13, 235), (14, 255), (15, 323), (16, 346), (17, 389), (18, 444), (19, 504), (20, 538), (21, 602), (22, 670), (23, 718), (24, 783), (25, 859), (26, 906), (27, 964), (28, 1046), (29, 1171), (30, 1193), (31, 1337), (32, 1436), (33, 1486), (34, 1627), (35, 1635), (36, 1747), (37, 1857), (38, 1971), (39, 2093), (40, 2174), (41, 2343), (42, 2398), (43, 2513), (44, 2638), (45, 2771), (46, 2952), (47, 3065), (48, 3127), (49, 3263)]


In [176]:
print([(n, Jacobi(MatrixGeneration.hilbert(n), eps=1e-3).run()[1]) for n in range(3, 50)])

[(3, 5), (4, 11), (5, 17), (6, 22), (7, 28), (8, 33), (9, 38), (10, 45), (11, 51), (12, 60), (13, 63), (14, 70), (15, 72), (16, 79), (17, 86), (18, 93), (19, 98), (20, 107), (21, 114), (22, 117), (23, 126), (24, 132), (25, 138), (26, 143), (27, 152), (28, 160), (29, 165), (30, 172), (31, 179), (32, 188), (33, 195), (34, 204), (35, 209), (36, 215), (37, 223), (38, 231), (39, 240), (40, 246), (41, 252), (42, 258), (43, 265), (44, 271), (45, 276), (46, 286), (47, 293), (48, 300), (49, 308)]


In [177]:
print([(n, Jacobi(MatrixGeneration.hilbert(n), eps=1e-6).run()[1]) for n in range(3, 50)])

[(3, 7), (4, 15), (5, 24), (6, 35), (7, 50), (8, 63), (9, 73), (10, 88), (11, 108), (12, 121), (13, 136), (14, 142), (15, 164), (16, 171), (17, 189), (18, 213), (19, 226), (20, 244), (21, 264), (22, 280), (23, 305), (24, 326), (25, 344), (26, 360), (27, 381), (28, 380), (29, 403), (30, 422), (31, 429), (32, 475), (33, 473), (34, 505), (35, 513), (36, 529), (37, 555), (38, 586), (39, 619), (40, 638), (41, 628), (42, 671), (43, 694), (44, 701), (45, 755), (46, 742), (47, 748), (48, 763), (49, 790)]
